In [1]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
from selenium import webdriver
import matplotlib.pyplot as plt
import time
import pandas as pd
import openpyxl
import os
import seaborn as sns
import plotly.express as px
import re

# web: https://portal.3gpp.org/Specifications.aspx
# date: 2023/3/8

### 1. 下載spec的parent-children關係 

In [ ]:
df = pd.read_excel("reference/Work_plan_3gpp_230110.xlsm")
SpecificationList_df = pd.read_excel("reference/2023-03-07_1805_SpecificationList_2e6c4e.xlsx")
ws = openpyxl.load_workbook('reference/2023-03-07_1805_SpecificationList_2e6c4e.xlsx')['Specifications']

relationships = {}

In [3]:
def get_spec_relationship(spec_link):
    driver.get(spec_link)
    time.sleep(0.5)
    related_btn = driver.find_elements(By.XPATH, '//*[@class="rtsTxt"]')[3]     
    related_btn.click()
    
    p_elements = driver.find_elements(By.XPATH, "//*[@id='parentSpecifications_specificationsGrid_ctl00']//tbody//tr")
    c_elements = driver.find_elements(By.XPATH, "//*[@id='childSpecifications_specificationsGrid_ctl00']//tbody//tr")
    
    parents = []
    children = []

    for i in p_elements:
        if i.get_attribute('class') != 'rgNoRecords':
            parents.append(i.find_elements(By.XPATH, "td")[0].text)

    for i in c_elements:
        if i.get_attribute('class') != 'rgNoRecords':
            children.append(i.find_elements(By.XPATH, "td")[0].text)
    
    return {
        'parents': parents,
        'children': children
    }

In [17]:
driver = webdriver.Chrome(ChromeDriverManager().install())


for i in range(1, ws.max_row - 1):
    cur_spec = ws.cell(1 + i, 1)
    if cur_spec.value not in relationships.keys():
        relationships[cur_spec.value] = get_spec_relationship(cur_spec.hyperlink.target)

In [1]:
df = pd.DataFrame.from_dict(relationships, orient='index')
df.to_excel('relationship_result.xlsx', encoding='utf_8_sig', index=True)

### 2. 整理表格－parents table & children table

In [2]:
rl_data = pd.read_excel("relationship_result.xlsx")

In [3]:
rl_data.replace(r"[\'\[\]]", '', regex=True, inplace=True)
rl_data.rename(columns={'Unnamed: 0': 'spec'}, inplace=True)

p_only = rl_data.assign(parents=rl_data.parents.str.split(", ")).explode("parents").drop(columns=['children'])
c_only = rl_data.assign(children=rl_data.children.str.split(", ")).explode("children").drop(columns=['parents'])
print('Length of parents only: ', len(p_only), '; Length of children only: ', len(c_only),)

Length of parents only:  4198 ; Length of children only:  4251


In [4]:
c_only.to_excel('relationship_result_children_only.xlsx', encoding='utf_8_sig', index=False)
p_only.to_excel('relationship_result_parents_only.xlsx', encoding='utf_8_sig', index=False)

C:\Users\Penny\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [5]:
# 合併children_only & parents_only之後刪除重複值
rl_data = pd.read_excel("relationship_result_cleaned.xlsx")
rl_data.drop_duplicates(inplace=True)
rl_data.to_excel('relationship_result_cleaned.xlsx', encoding='utf_8_sig', index=False)

### 3. 將spec關係轉換成公司名稱呈現

In [3]:
child_df = pd.read_excel("relationship_result_children_only.xlsx")
spec_df = pd.read_excel("reference/spec_result_all_cleaned.xlsx")

In [50]:
child_df

,parent,children
0,00.01U,NaN
1,00.02,NaN
2,00.02U,NaN
3,01.00,21.801
4,01.00,21.100
...,...,...
4246,TBR 5,NaN
4247,TBR 9,NaN
4248,TBR19,NaN
4249,TBR20,NaN


In [70]:
spec_by_company = pd.DataFrame()

for index, row in child_df.iterrows():
    if not pd.isna(row['children']):
#         print(spec_df.loc[spec_df['Spec No'] == row['children']]['Primary rapporteur company'])
        try:
            parent_cmp = spec_df.loc[spec_df['Spec No'] == row['parent']]['Primary rapporteur company'].values[0]
        except:
            parent_cmp = ''
        try:
            child_cmp = spec_df.loc[spec_df['Spec No'] == row['children']]['Primary rapporteur company'].values[0]
        except:
            child_cmp = ''

        if parent_cmp != 'waiting for new company' and child_cmp != 'waiting for new company':
            result_row = pd.DataFrame({
                'parent': parent_cmp,
                'children': child_cmp,
            }, index=[0])
            spec_by_company = pd.concat([spec_by_company, result_row], axis=0)

In [71]:
spec_by_company

,parent,children
0,hillebrand gmbh,etsi
0,hillebrand gmbh,etsi
0,etsi,etsi
0,etsi,etsi
0,home office,etsi
...,...,...
0,etsi,etsi
0,gemalto n.v.,g+d ms
0,gemalto n.v.,qualcomm
0,morpho cards gmbh,morpho cards gmbh


In [73]:
spec_by_company.to_excel('relationship_by_company.xlsx', encoding='utf_8_sig', index=False)